# **Install ktrain and tensorflow gpu**

In [ ]:
!pip install ktrain

In [ ]:
!pip install tensorflow-gpu

# **Import packages**

In [ ]:
import pandas as pd
import ktrain
from ktrain import text
from sklearn.model_selection import train_test_split


In [ ]:
import tensorflow as tf
from tensorflow import keras


# **Read dataset as dataframe**

In [ ]:
df = pd.read_csv('/content/drive/My Drive/goemotions_aug_dairai_train_cleaned.csv')

In [ ]:
df = df[:100000]

In [ ]:
df.head()

,Unnamed: 0,anger,augmented,cleaned_processed,datasource,fear,joy,sadness
0,206509,0.0,0,i feel that some korea guy are handsome and so...,dairai,0.0,1.0,0.0
1,375242,0.0,0,i put my pen to paper and made a list of thing...,dairai,0.0,0.0,0.0
2,166570,1.0,0,i wish i only had to feel the pain of the pett...,dairai,0.0,0.0,0.0
3,200580,0.0,0,i feel passionate about this journey and stand...,dairai,0.0,1.0,0.0
4,300766,0.0,0,i feel like i have convinced myself of these f...,dairai,0.0,1.0,0.0


In [ ]:
label = ['anger', 'fear', 'joy', 'sadness']

In [ ]:
df = df.rename(columns={'cleaned_processed': 'text'})

# **split the sub-dataset as trn and val**

In [ ]:
train, test = train_test_split(df, test_size=0.33, random_state=42)

In [ ]:
train_x = train['text']

In [ ]:
train_y = train[label]

In [ ]:
test_x = test['text']

In [ ]:
test_y = test[label]

# **get the model: albert-base-v2**


1.   use text.transformer to get the transformer instance
2.   preprocess train and test dataset, which will automaticly encode the text, and recognize if it is a multi-label task. the trn and val are made to fit the model you given.
3.   get the classifier(the real model)
4.   wrap it up into a ktrain learner
5.   use the ktrain learner to train the model



In [ ]:
MODEL_NAME = 'albert-base-v2'

In [ ]:
t = text.Transformer(MODEL_NAME, maxlen=50, class_names=label)

In [ ]:
trn = t.preprocess_train(train_x.values, train_y.values)

preprocessing train...
language: en
train sequence lengths:
	mean : 20
	95percentile : 42
	99percentile : 53


Is Multi-Label? True


In [ ]:
val = t.preprocess_test(test_x.values, test_y.values)

preprocessing test...
language: en
test sequence lengths:
	mean : 20
	95percentile : 42
	99percentile : 54


In [ ]:
model = t.get_classifier()

In [ ]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs/', write_graph=True, write_images=True)

In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)

In [ ]:
learner.fit_onecycle(3e-5, 5, callbacks=[tbCallBack])



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/5
4188/4188 [==============================] - 627s 150ms/step - loss: 0.1665 - accuracy: 0.7414 - val_loss: 0.0702 - val_accuracy: 0.8120
Epoch 2/5
4188/4188 [==============================] - 622s 148ms/step - loss: 0.0700 - accuracy: 0.8134 - val_loss: 0.0621 - val_accuracy: 0.8165
Epoch 3/5
4188/4188 [==============================] - 621s 148ms/step - loss: 0.0600 - accuracy: 0.8150 - val_loss: 0.0579 - val_accuracy: 0.8075
Epoch 4/5
4188/4188 [==============================] - 620s 148ms/step - loss: 0.0471 - accuracy: 0.8183 - val_loss: 0.0508 - val_accuracy: 0.8223
Epoch 5/5
4188/4188 [==============================] - 619s 148ms/step - loss: 0.0318 - accuracy: 0.8219 - val_loss: 0.0536 - val_accuracy: 0.8182


In [ ]:
model.summary()

Model: "tf_albert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  11683584  
_________________________________________________________________
dropout_9 (Dropout)          multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 11,686,660
Trainable params: 11,686,660
Non-trainable params: 0
_________________________________________________________________


# **Save Model as a predictor**

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
predictor.predict('love surface love lavender')

[('anger', 0.0013034324),
 ('fear', 0.00096694403),
 ('joy', 0.9978543),
 ('sadness', 0.00090368226)]

In [ ]:
predictor.save('/content/drive/My Drive/albert_model_on_first100000_goemotions')

# **load model and predict**
if you want to train it again, use predictor.model to get the model, and wrap it into a ktrain learner, continue the trian process

In [ ]:
p = ktrain.load_predictor('/content/drive/My Drive/albert_model_on_first100000_goemotions')

In [ ]:
p.predict('how are you BERT, why are you so training so slow')

[('anger', 0.25451094),
 ('fear', 0.4445131),
 ('joy', 0.013134955),
 ('sadness', 0.08884486)]

In [ ]:
p.predict('bert is so slow, it annoys me a lot')

[('anger', 0.9948118),
 ('fear', 0.00075041334),
 ('joy', 0.001371897),
 ('sadness', 0.0020669135)]

In [ ]:
p.predict('you are always late, never do that again')

[('anger', 0.30835918),
 ('fear', 0.32899058),
 ('joy', 0.012022821),
 ('sadness', 0.3909705)]

In [ ]:
p.predict('I will be to school tomorrow')

[('anger', 0.5374949),
 ('fear', 0.044114694),
 ('joy', 0.09671777),
 ('sadness', 0.454767)]